In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'		   FINAL_DATASET_v2   Signs-MNIST-tiny
 divya_poluri_resume_walmart.pdf  'My Drive'	      yolov3
 EGOHANDS-yolov3		   Sample_v1
'(FAILED)CUSTOM_YOLOv3_v1'	   SEM7-PROJECT


**1) Clone the Darknet**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14358, done.
remote: Total 14358 (delta 0), reused 0 (delta 0), pack-reused 14358
Receiving objects: 100% (14358/14358), 13.06 MiB | 23.71 MiB/s, done.
Resolving deltas: 100% (9765/9765), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 54000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=96@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=96@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=96@' cfg/yolov3_training.cfg

In [ ]:
!echo "hands" > data/obj.names
!echo -e 'classes= 27\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-10-30 10:50:22--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   241KB/s    in 11m 43s 

2020-10-30 11:02:07 (226 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/ASL_VIDEO_7_1003.txt  
  inflating: data/obj/ASL_VIDEO_7_1005.jpg  
  inflating: data/obj/ASL_VIDEO_7_1005.txt  
  inflating: data/obj/ASL_VIDEO_7_1020.jpg  
  inflating: data/obj/ASL_VIDEO_7_1020.txt  
  inflating: data/obj/ASL_VIDEO_7_1021.jpg  
  inflating: data/obj/ASL_VIDEO_7_1021.txt  
  inflating: data/obj/ASL_VIDEO_7_1022.jpg  
  inflating: data/obj/ASL_VIDEO_7_1022.txt  
  inflating: data/obj/ASL_VIDEO_7_1038.jpg  
  inflating: data/obj/ASL_VIDEO_7_1038.txt  
  inflating: data/obj/ASL_VIDEO_7_1039.jpg  
  inflating: data/obj/ASL_VIDEO_7_1039.txt  
  inflating: data/obj/ASL_VIDEO_7_1054.jpg  
  inflating: data/obj/ASL_VIDEO_7_1054.txt  
  inflating: data/obj/ASL_VIDEO_7_1056.jpg  
  inflating: data/obj/ASL_VIDEO_7_1056.txt  
  inflating: data/obj/ASL_VIDEO_7_1071.jpg  
  inflating: data/obj/ASL_VIDEO_7_1071.txt  
  inflating: data/obj/ASL_VIDEO_7_1072.jpg  
  inflating: data/obj/ASL_VIDEO_7_1072.txt  
  in

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/ASL_VIDEO_9_1156.jpg', 'data/obj/ASL_VIDEO_9_680.jpg', 'data/obj/ASL_VIDEO_0_600.jpg', 'data/obj/M148.jpg', 'data/obj/Q18.jpg', 'data/obj/R130.jpg', 'data/obj/Y21.jpg', 'data/obj/W20_mad.jpg', 'data/obj/C6_mad.jpg', 'data/obj/ASL_VIDEO_2_535.jpg', 'data/obj/X117.jpg', 'data/obj/N13.jpg', 'data/obj/A17_mad.jpg', 'data/obj/ASL_VIDEO_0_1308.jpg', 'data/obj/ASL_VIDEO_10_74.jpg', 'data/obj/X13_mad.jpg', 'data/obj/ASL_VIDEO_10_668.jpg', 'data/obj/ASL_VIDEO_10_717.jpg', 'data/obj/ASL_VIDEO_9_53.jpg', 'data/obj/G5.jpg', 'data/obj/ASL_VIDEO_2_554.jpg', 'data/obj/ASL_VIDEO_2_910.jpg', 'data/obj/W17.jpg', 'data/obj/O11_mad.jpg', 'data/obj/ASL_VIDEO_0_1236.jpg', 'data/obj/T109.jpg', 'data/obj/ASL_VIDEO_8_342.jpg', 'data/obj/A19_mad.jpg', 'data/obj/ASL_VIDEO_10_547.jpg', 'data/obj/ASL_VIDEO_0_360.jpg', 'data/obj/Q42.jpg', 'data/obj/ASL_VIDEO_0_1536.jpg', 'data/obj/E137.jpg', 'data/obj/ASL_VIDEO_7_732.jpg', 'data/obj/C30_mad.jpg', 'data/obj/F17.jpg', 'data/obj/Y49.jpg', 'data/obj/ASL_VIDE

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 113224, rewritten_bbox = 0.000000 % 

 1828: 0.219006, 0.231886 avg loss, 0.001000 rate, 5.287690 seconds, 116992 images, 85.092926 hours left
Loaded: 0.000041 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.679279, GIOU: 0.676764), Class: 0.993983, Obj: 0.672275, No Obj: 0.002009, .5R: 1.000000, .75R: 0.000000, count: 2, class_loss = 0.261661, iou_loss = 0.141285, total_loss = 0.402946 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.708753, GIOU: 0.682696), Class: 0.854501, Obj: 0.650992, No Obj: 0.000401, .5R: 1.000000, .75R: 0.500000, count: 2, class_loss = 0.189747, iou_loss = 0.134492, total_loss = 0.324240 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 tot

# **Continued Training**



In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show